In [30]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [31]:
df_train = pd.read_csv("../data/raw/train.csv")
df_test = pd.read_csv("../data/raw/test.csv")

### Rename columns

In [32]:
# lower case column names
df_train.columns = df_train.columns.str.lower()
df_test.columns = df_test.columns.str.lower()

In [33]:
# rename columns to pythonic names
df_train = df_train.rename(columns={
    'passengerid': 'passenger_id',
    'survived': 'survived',
    'pclass': 'passenger_class',
    'sibsp': 'sibling_spouse_count',
    'parch': 'parent_child_count',
    'ticket': 'ticket_number',
    'embarked': 'embarked_port',
})

df_test = df_test.rename(columns={
    'passengerid': 'passenger_id',
    'survived': 'survived',
    'pclass': 'passenger_class',
    'sibsp': 'sibling_spouse_count',
    'parch': 'parent_child_count',
    'ticket': 'ticket_number',
    'embarked': 'embarked_port',
})

### Define unnecessary columns

In [34]:
cols_to_drop =['name', 'cabin', 'ticket_number', 'passenger_id']

In [35]:
# check column names
display(df_train.head())
display(df_test.head())

,passenger_id,survived,passenger_class,name,sex,age,sibling_spouse_count,parent_child_count,ticket_number,fare,cabin,embarked_port
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


,passenger_id,passenger_class,name,sex,age,sibling_spouse_count,parent_child_count,ticket_number,fare,cabin,embarked_port
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


### Impute embarked values
Only two missing values, filled in with most common value

In [36]:
df_train["embarked_port"] = df_train["embarked_port"].fillna(df_train["embarked_port"].mode()[0])
df_test["embarked_port"] = df_test["embarked_port"].fillna(df_test["embarked_port"].mode()[0])

In [37]:
# check missing values
display(df_train.isnull().sum())
display(df_test.isnull().sum())

passenger_id              0
survived                  0
passenger_class           0
name                      0
sex                       0
age                     177
sibling_spouse_count      0
parent_child_count        0
ticket_number             0
fare                      0
cabin                   687
embarked_port             0
dtype: int64

passenger_id              0
passenger_class           0
name                      0
sex                       0
age                      86
sibling_spouse_count      0
parent_child_count        0
ticket_number             0
fare                      1
cabin                   327
embarked_port             0
dtype: int64

### Impute age values
Check correlation with age, grouped by columns with high correlation

In [38]:
# fill missing values in age column with grouped median
df_train["age"] = df_train["age"].fillna(df_train.groupby(["passenger_class", "sibling_spouse_count", "parent_child_count"])["age"].transform("median"))
df_test["age"] = df_test["age"].fillna(df_test.groupby(["passenger_class", "sibling_spouse_count", "parent_child_count"])["age"].transform("median"))
# fallback for remaining missing values fill with global age median
df_train["age"] = df_train["age"].fillna(df_train["age"].median())
df_test["age"] = df_test["age"].fillna(df_test["age"].median())

In [39]:
# check missing values
display(df_train['age'].isnull().sum())
display(df_test['age'].isnull().sum())

0

0

In [40]:
df_train.to_csv('../data/processed/train_cleaned.csv', index=False)
df_test.to_csv('../data/processed/test_cleaned.csv', index=False)